In [5]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from scipy.stats import randint
import joblib  # Modeli ve özellikleri kaydetmek için

# 📌 1. Eğitim Verisini Hazırla
def preprocess_data(file_path, is_train=True):
    df = pd.read_csv(file_path)
    print(f"✅ Veri yüklendi: {df.shape}")

    # 📌 Kategorik ve Sayısal Değişkenleri Ayır
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

    # 📌 Eksik değerleri doldurma
    num_imputer = SimpleImputer(strategy="median")
    cat_imputer = SimpleImputer(strategy="most_frequent")

    if numerical_cols:
        df[numerical_cols] = num_imputer.fit_transform(df[numerical_cols])

    if categorical_cols:
        df[categorical_cols] = cat_imputer.fit_transform(df[categorical_cols].astype(str))

    print("✅ Eksik değerler dolduruldu!")

    # 📌 One-Hot Encoding
    encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    
    if is_train:
        df_encoded = pd.DataFrame(encoder.fit_transform(df[categorical_cols]))
        joblib.dump(encoder, "encoder.pkl")  # Eğitilen encoder'ı kaydet
    else:
        encoder = joblib.load("encoder.pkl")  # Önceki encoder'ı yükle
        df_encoded = pd.DataFrame(encoder.transform(df[categorical_cols]))

    df_encoded.columns = encoder.get_feature_names_out(categorical_cols)
    df = df.drop(columns=categorical_cols).reset_index(drop=True)
    df = pd.concat([df, df_encoded], axis=1)

    print("✅ One-Hot Encoding tamamlandı!")

    # 📌 Train seti için hedef değişkeni ayır
    if is_train:
        y = df["rainfall"]
        X = df.drop(columns=["rainfall", "id"])
        joblib.dump(X.columns, "train_features.pkl")  # Kullanılan feature'ları kaydet
        return X, y
    else:
        test_ids = df["id"]
        X = df.drop(columns=["id"])

        # 📌 Feature Sıralamasını Düzelt
        train_features = joblib.load("train_features.pkl")  # Train'de kullanılan sütunları al
        for col in train_features:
            if col not in X.columns:
                X[col] = 0  # Eksik sütun varsa sıfırla
        X = X[train_features]  # Sütun sırasını düzelt

        print("✅ Test seti train ile uyumlu hale getirildi!")
        return X, test_ids

# 📌 2. Random Forest Modelini Eğitme
def train_optimized_random_forest(X_train, y_train):
    param_grid = {
        "n_estimators": randint(200, 500),
        "max_depth": randint(10, 20),
        "min_samples_split": randint(2, 10),
        "min_samples_leaf": randint(1, 5),
        "max_features": ["sqrt", "log2"]
    }

    rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)
    search = RandomizedSearchCV(rf_model, param_grid, n_iter=15, scoring="roc_auc", cv=3, verbose=1, n_jobs=-1, random_state=42)
    search.fit(X_train, y_train)

    best_model = search.best_estimator_
    joblib.dump(best_model, "random_forest_model.pkl")  # Modeli kaydet
    print(f"✅ En iyi Random Forest Modeli bulundu: {search.best_params_}")
    return best_model

# 📌 3. Test Verisinde Tahmin Yapma ve Sonucu Kaydetme
def predict_and_save(model, test_file, output_file="submission_rf_fixed.csv"):
    X_test, test_ids = preprocess_data(test_file, is_train=False)
    y_pred = model.predict_proba(X_test)[:, 1]  # Yağmur olma olasılığını al
    submission = pd.DataFrame({"id": test_ids, "rainfall": y_pred})
    submission.to_csv(output_file, index=False)
    print(f"✅ Tahminler {output_file} dosyasına kaydedildi!")

# 📌 Ana Akış
if __name__ == "__main__":
    train_file = "train.csv"
    test_file = "test.csv"

    # 📌 Veriyi yükle ve işle
    X_train, y_train = preprocess_data(train_file, is_train=True)

    # 📌 Modeli eğit
    best_rf_model = train_optimized_random_forest(X_train, y_train)

    # 📌 Test verisi üzerinde tahmin yap ve sonucu kaydet
    predict_and_save(best_rf_model, test_file)


✅ Veri yüklendi: (2190, 13)
✅ Eksik değerler dolduruldu!
✅ One-Hot Encoding tamamlandı!
Fitting 3 folds for each of 15 candidates, totalling 45 fits
✅ En iyi Random Forest Modeli bulundu: {'max_depth': 11, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 6, 'n_estimators': 360}
✅ Veri yüklendi: (730, 12)
✅ Eksik değerler dolduruldu!
✅ One-Hot Encoding tamamlandı!
✅ Test seti train ile uyumlu hale getirildi!
✅ Tahminler submission_rf_fixed.csv dosyasına kaydedildi!
